## 기초통계량

- mode
- median
- mean
- variance
- standard deviation
- skewness
- kurtosis

In [ ]:
import json
#key:종목 / value:symbol인 딕셔너리 가져오기
stock2code = json.load(open('data/stock2code.json','r'))

naver = stock2code['NAVER']

import FinanceDataReader as fdr
data = fdr.DataReader(naver)
data

In [ ]:
#data의 간단한 통계정보 확인
data.describe()

In [ ]:
print("****MEDIAN****")
print(data.median())
print()

print("****산술평균****")
print(data.mean())
print()
print("****산술평균(axis=1)****")
print(data[['Open','High','Low','Close']].mean(axis=1))
print()
print("****최빈값****")
print(data.mode())
print()
print("****분산****")
print(data.var())
print()
print("****표준편차****")
print(data.std())
print()
print("****왜도****")
print(data.skew())
print()
print("****첨도****")
print(data.kurtosis())

### 기하평균

In [ ]:
#산술평균
data.Change.loc['2021-06-01':'2021-06-30'].mean()

In [ ]:
len(data.Change.loc['2021-06-01':'2021-06-30'])

In [ ]:
#기하평균
((data.Change.loc['2021-06-01':'2021-06-30'] + 1).prod()) ** (1/22)  - 1

In [ ]:
def geo_mean(x):
    return (x +1).prod() ** (1/len(x)) - 1

In [ ]:
print("기하평균", geo_mean(data.Change))
print("산술평균", data.Change.mean())

## 이동평균

### 단순이동평균과 지수이동평균

In [ ]:
data.Close['2021-05-01':].plot(label='close')
data.Close.rolling(12).mean().loc['2021-05-01':].plot(label='sma12')
data.Close.ewm(12).mean().loc['2021-05-01':].plot(label='ema12')
plt.legend()
plt.show()

## 상관

- 공분산
- 상관계수

In [ ]:
df1 = fdr.DataReader(stock2code['셀트리온']).Close
df2 = fdr.DataReader(stock2code['SK바이오사이언스']).Close
df = pd.concat([df1, df2], axis=1).dropna()
df.columns = ['Cellt','SKBio']

#공분산
df.cov()

In [ ]:
#상관계수
df.corr()

In [ ]:
#공분산 값과 표준편차값으로 상관계수 연산
df.cov().iloc[0,1] / (df.Cellt.std() * df.SKBio.std())

## 회귀분석

In [ ]:
!pip install scipy
!pip install scikit-learn
!pip install statsmodels

### scipy

In [ ]:
import scipy as sp

In [ ]:
data = data.reset_index()[['Close']]

#회귀선 기울기, 상수항
slope, intercept, _, _, _, = sp.stats.linregress(data.index, data.Close)
print(slope, intercept)

In [ ]:
plt.plot(data.index, data.Close)
plt.plot(data.index, data.index * slope + intercept)
plt.show()

### scikit-learn

In [ ]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [ ]:
#상수항 넣어주기
index= sm.add_constant(data.index)
index

In [ ]:
model = LinearRegression(fit_intercept=True)
model.fit(index, data.Close)

In [ ]:
#회귀선 기울기, 상수항
model.coef_, model.intercept_

In [ ]:
plt.plot(data.index, data.Close)
plt.plot(index, index * model.coef_ + model.intercept_)
plt.show()

### statsmodels

In [ ]:
index = sm.add_constant(data.index)
model = sm.OLS(data.Close, index)
result = model.fit()
result.summary()

In [ ]:
result.params

In [ ]:
result.rsquared

In [ ]:
plt.plot(data.index, data.Close)
result.fittedvalues.plot()
plt.show()